In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import qpfas

In [ ]:
exp_df = pd.read_json("Results/output_1a.json")
print(exp_df.columns)
print(exp_df.shape)
print(exp_df.iloc[0])

In [ ]:
exp_df[["n_qubits", "n_qubits_taper", "molecule", "ansatz_name", "ansatz_depth", "active_space_method", "active_space"]]

In [ ]:
def nice_name(x):
    num_idx = [i for i in range(len(x)) if x[i].isnumeric()]
    for c, i in enumerate(num_idx):
        x = x[:i+c] + "_" + x[i+c:]
    x = "$" + x + "$"
    return x

In [ ]:
full_active_name = {"None": "Full", "frozen_core": "Frozen Core", "noons_0.002":"NOONs (0.02)"}

In [ ]:
benchmark_method = "ccsd"

# for this plot we don't care about the ansatz, so we just use the hardware ansatz results
exp_df_sub = exp_df[exp_df["ansatz_name"] == "hardware"]
print(exp_df_sub[["molecule", "active_space_method"]])


mols = [i["molecule"] for _, i in exp_df_sub[exp_df_sub["active_space_method"] == "None"].iterrows()]
print(mols)

width = 0.2
rs = np.arange(len(mols))

fig, axs = plt.subplots(nrows=3, sharex="col", figsize=(8.4, 5))


# ax 1: error
exact_energy = exp_df_sub[exp_df_sub["active_space_method"]=="None"][benchmark_method + "_energy"].values
print(exact_energy)

for c, i in enumerate(["None", "frozen_core", "noons_0.002"]):
    """
    1) Error in Energy
    """
    # hf error
    if i == "None":
        hf_err = exp_df_sub[exp_df_sub["active_space_method"]==i]["hf_energy"].values - exact_energy
        axs[0].bar(rs-width, hf_err, width=width, label='HF', color='C3', edgecolor='k')

    else:
        err = exp_df_sub[exp_df_sub["active_space_method"]==i][benchmark_method + "_energy"].values - exact_energy
        axs[0].bar(rs+width*(c-1), err, width=width, label=full_active_name[i], color='C%i' % c, edgecolor='k')
        print(err)
    """
    2) Number of Qubits    
    """
    n_qubits = exp_df_sub[exp_df_sub["active_space_method"]==i]["n_qubits"].values
    if i == "None":
        axs[1].bar(rs+width*(c-1), n_qubits, width=width, color='C%i' % c, edgecolor='k', label=full_active_name[i])
    else:
        axs[1].bar(rs+width*(c-1), n_qubits, width=width, color='C%i' % c, edgecolor='k')

    axs[1].bar(rs+width*(c-1), exp_df_sub[exp_df_sub["active_space_method"]==i]["n_qubits_taper"].values, width=width, color='C%i' % c, hatch="////", edgecolor='k')
    
    """
    3) Number of Ham. Terms    
    """
    n_terms = exp_df_sub[exp_df_sub["active_space_method"]==i]["n_hamiltonian_terms"].values
    axs[2].bar(rs+width*(c-1), n_terms, width=width, color='C%i' % c, edgecolor='k')
    axs[2].bar(rs+width*(c-1), exp_df_sub[exp_df_sub["active_space_method"]==i]["n_hamiltonian_terms_taper"].values, width=width, color='C%i' % c, hatch="////", edgecolor='k')

axs[0].set_ylabel("Error (H)")
axs[1].set_ylabel("Num. of Qubits")
axs[2].set_ylabel("Num. of Terms")

axs[0].legend()
axs[1].legend()

#axs[0].set_yscale("log")

axs[2].set_yscale("log")
names = [nice_name(a) for a in mols]
plt.xticks(rs, names)
plt.savefig("Results/results1_psi4.pdf")
plt.show()



In [ ]:
mol = "CH4"

gates = ["gate_total", "gate_cx", "gate_rx", "gate_ry", "gate_rz"]

width = 0.1

fig, axs = plt.subplots(nrows=2, sharex='col')

for c1, ansatz_method in enumerate(["uccsd", "hardware"]):
    df_sub = exp_df[(exp_df["molecule"] == mol) & (exp_df["ansatz_name"] == ansatz_method)]
    as_method = df_sub["active_space_method"]
    print(as_method)
        
    xs = np.arange(df_sub.shape[0])
    for c2, gate_type in enumerate(gates):
        axs[c1].bar(xs+c2*width, df_sub[gate_type], width=width, label=gate_type.split("_")[1], color="C%i" % c2, edgecolor='k')
        
    if ansatz_method == "hardware":
        #Get tapering circuit data
        circuit_stats = [qpfas.chemistry.hardware_ansatz_gate_stats(i, 2) for i in df_sub["n_qubits_taper"].values]
        df_taper = pd.DataFrame(circuit_stats)
        df_taper["gate_rx"] = np.NaN
        
        for c2, gate_type in enumerate(gates):
            axs[c1].bar(xs+c2*width, df_taper[gate_type], width=width, hatch="////", color="C%i" % c2, edgecolor='k')

    axs[c1].set_yscale("log")
    axs[c1].set_ylabel("Gate Count (%s)" % ansatz_method)
    
axs[0].legend()

axs[1].set_xticks(xs+width*(len(gates)-1)*0.5)

xlabel = [full_active_name[i] for i in as_method]
axs[1].set_xticklabels(xlabel, fontsize=8)
axs[1].set_xlabel("Active Space")

#fig.suptitle(nice_name(mol))

#plt.savefig("Results/results2_%s.pdf" % mol)
#plt.show()

In [ ]:
for ansatz_method in ["uccsd", "hardware"]:
    df_sub = exp_df[exp_df["ansatz_name"] == ansatz_method]
    plt.scatter(df_sub["n_qubits"], df_sub["gate_total"], label=ansatz_method)

xs = np.linspace(4, 20, 100)
ys = [qpfas.chemistry.hardware_ansatz_gate_stats(i, 2)["gate_total"] for i in xs]
plt.plot(xs, ys, c="C1", ls='--')


plt.yscale("log")

plt.xlabel("Num. of Qubits")
plt.ylabel("Total Gate Count")

plt.legend()

#plt.savefig("Results/results3.pdf")
